In [79]:
from tika import parser
import os
from os import listdir
from os.path import isfile, join
import re
from collections import defaultdict
import time
from datetime import date, datetime
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
matplotlib.use("TkAgg")
import tkinter.scrolledtext as tkst
import numpy as np
from datetime import timedelta

from tkinter import *
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

path = "process-pdf/"
SNP = {'^GSPC': 0}

In [25]:
def retriveFiles(path):
    return [join(path, f) for f in listdir(path) if isfile(join(path, f))]

In [26]:
files = retriveFiles(path)

In [27]:
def generateDates(files):
    root = len(path)
    res = []
    for file in files:    
        dstr = file[root:root+8]
        d = date(int(dstr[0:4]),int(dstr[4:6]),int(dstr[6:8]))
        res.append(d)
    return res

In [39]:


# purchaseDate = 'YYYYMMDD'
# threeStarVolume, fourStarVolume, fiveStarVolume
# trailingTime

root = Tk()
root.geometry('1000x600')
root.title("Morningstar Strategy Simulator")
root.resizable(False, False)

lf_dataRange = LabelFrame(root, text = "Data Range")
lf_dataRange.grid(row = 0, column = 0, columnspan = 2, padx = 5, pady = 5)

# awaiting filename data
# generateDates(files)
dataRange = generateDates(files)
dataRangeSelect = ttk.Combobox(lf_dataRange, state="readonly", values = dataRange)
dataRangeSelect.pack()
dataRangeSelect.current(1)
def callbackFunc(event):
    print("New Element Selected", dataRangeSelect.get())
    day = ''.join(str(dataRangeSelect.get()).split('-'))
    print (day)
    if (day == ''):
        day = '20180223'
    report_date = date(int(day[0:4]),int(day[4:6]),int(day[6:8]))
dataRangeSelect.bind("<<ComboboxSelected>>", callbackFunc)

# lf_purchaseDate = LabelFrame(root, text = "Purchase Date (YYYYMMDD)")
# lf_purchaseDate.grid(row = 1, column = 0, columnspan = 2, padx = 5, pady = 5)
# entry_purchaseDate = Entry(lf_purchaseDate, bd = 2)
# entry_purchaseDate.pack()
# day = str(entry_purchaseDate.get())
# if (day == ''):
#     day = '20180808'
# report_date = date(int(day[0:4]),int(day[4:6]),int(day[6:8]))

lf_threeStarVolume = LabelFrame(root, text = "3-star Quantity")
lf_threeStarVolume.grid(row = 0, column = 2, columnspan = 2, padx = 5, pady = 5)
entry_threeStarVolume = Entry(lf_threeStarVolume, bd = 2)
entry_threeStarVolume.pack()
threeStarVolume = entry_threeStarVolume.get()

lf_fourStarVolume = LabelFrame(root, text = "4-star Quantity")
lf_fourStarVolume.grid(row = 1, column = 2, columnspan = 2, padx = 5, pady = 5)
entry_fourStarVolume = Entry(lf_fourStarVolume, bd = 2)
entry_fourStarVolume.pack()
fourStarVolume = entry_fourStarVolume.get()
    
lf_fiveStarVolume = LabelFrame(root, text = "5-star Quantity")
lf_fiveStarVolume.grid(row = 2, column = 2, columnspan = 2, padx = 5, pady = 5)
entry_fiveStarVolume = Entry(lf_fiveStarVolume, bd = 2)
entry_fiveStarVolume.pack()
fiveStarVolume = entry_fiveStarVolume.get()

lf_TrailingTime = LabelFrame(root, text = "Trailing Return Days")
lf_TrailingTime.grid(row = 0, rowspan = 2, column = 4, columnspan = 2, padx = 5, pady = 5, sticky = "N" + "S")
MODES = [
    ("5 Days", 5),
    ("30 Days", 30),
    ("91 Days", 91),
    ("180 Days", 180),
]

v = StringVar()
v.set(5)

for text, mode in MODES:
    b = Radiobutton(lf_TrailingTime, text = text, variable = v, value = mode)
    b.pack(anchor = W)
trailingTime = int(v.get())
end_date = report_date + timedelta(days=trailingTime)

btn_Submit = Button(root, text = "Submit")
btn_Submit.grid(row = 1, column = 6, columnspan = 2, padx = 5, pady = 5, sticky = "N" + "S" + "E" + "W")

# still need to implement submit func

# function callback for plt
f = Figure(figsize=(5,5), dpi=100)
a = f.add_subplot(111)
a.plot([1,2,3,4,5,6,7,8],[5,6,1,3,8,9,3,5])

canvas = FigureCanvasTkAgg(f, root)
canvas.draw()
canvas.get_tk_widget().grid(row = 3, rowspan = 6, column = 3, columnspan = 6, sticky = "N" + "S" + "E" + "W")

tickerInfo = tkst.ScrolledText(
    master = root,
    wrap   = WORD,
    width  = 20,
    height = 10
)

tickerInfo.grid(row = 3, rowspan = 5, column = 0, columnspan = 3, sticky = "N" + "S" + "E" + "W")

# replace text with a func callback
tickerInfo.insert(INSERT,
"""\
Integer posuere erat a ante venenatis dapibus.
Posuere velit aliquet.
Aenean eu leo quam. Pellentesque ornare sem.
Lacinia quam venenatis vestibulum.
Nulla vitae elit libero, a pharetra augue.
Cum sociis natoque penatibus et magnis dis.
Parturient montes, nascetur ridiculus mus.
Integer posuere erat a ante venenatis dapibus.
Posuere velit aliquet.
Aenean eu leo quam. Pellentesque ornare sem.
Lacinia quam venenatis vestibulum.
Nulla vitae elit libero, a pharetra augue.
Cum sociis natoque penatibus et magnis dis.

Integer posuere erat a ante venenatis dapibus.
Posuere velit aliquet.
Aenean eu leo quam. Pellentesque ornare sem.
Lacinia quam venenatis vestibulum.
Nulla vitae elit libero, a pharetra augue.
Cum sociis natoque penatibus et magnis dis.

""")

tickerInfo.config(state = DISABLED)

for col in range(10):
    root.grid_columnconfigure(col, minsize = 100, pad = 2)

for row in range(10):
    root.grid_rowconfigure(row, minsize = 60, pad = 2)


while True:
    try:
        root.mainloop()
        break
    except UnicodeDecodeError:
        pass

New Element Selected 2019-01-11
20190111


In [40]:
def retrieveRatings(files, day):
    for file in files:
        if day in file:
            break
    raw = parser.from_file(file)
    pattern = '\(.+\) Q+'
    data_pairs = re.findall(pattern, raw['content'])
    res = defaultdict(int)
    for data in data_pairs:
        pair = data.split(' ');
        res[pair[0][1:-1]] = len(pair[1])
        
    return res

In [42]:
ticker_rating_pair = retrieveRatings(files, day)

In [43]:
#get open and close prices from yahoo api
#return dic {ticker: [rating, [price1, price2, ...]], ...}
def getTickerPrices(report_date, end_date, ticker_rating_pair):
    res = defaultdict(list)
    for ticker, rating in ticker_rating_pair.items():
        try:    
            price_info = data.DataReader(ticker, 'yahoo', report_date, end_date)
            prices = []
            for i in range (price_info.shape[0]):
                prices.append(price_info.iloc[i]['Close'])
            res[ticker] = [rating, prices]
        except:
            print(ticker, " is not public traded in North America stock market, ignored...", sep = "")
    return res

In [67]:
ticker_prices = getTickerPrices(report_date, end_date, ticker_rating_pair)
snp_prices = getTickerPrices(report_date, end_date, SNP)

ticker_prices

BT.A is not public traded in North America stock market, ignored...
941 is not public traded in North America stock market, ignored...
TLS is not public traded in North America stock market, ignored...
BMW is not public traded in North America stock market, ignored...
ATM is not public traded in North America stock market, ignored...
IMB is not public traded in North America stock market, ignored...
4452 is not public traded in North America stock market, ignored...
RB. is not public traded in North America stock market, ignored...
RDS.B is not public traded in North America stock market, ignored...
WPL is not public traded in North America stock market, ignored...
601288 is not public traded in North America stock market, ignored...
CSGN is not public traded in North America stock market, ignored...
8306 is not public traded in North America stock market, ignored...
PDL is not public traded in North America stock market, ignored...
RHC is not public traded in North America stock marke

defaultdict(list,
            {'CCJ': [5,
              [11.109999656677246,
               11.039999961853027,
               10.770000457763672,
               10.470000267028809,
               10.739999771118164]],
             'CMP': [4,
              [65.05000305175781,
               65.4000015258789,
               63.29999923706055,
               61.79999923706055,
               61.849998474121094]],
             'MLM': [4,
              [205.44000244140625,
               203.52000427246094,
               200.1300048828125,
               201.0,
               202.75]],
             'CMCSA': [4,
              [35.04999923706055,
               35.4900016784668,
               35.08000183105469,
               35.040000915527344,
               35.38999938964844]],
             'TEF': [5,
              [8.890000343322754,
               8.770000457763672,
               8.529999732971191,
               8.510000228881836,
               8.479999542236328]],
             'BA

In [71]:
#calculate daily profits
#return list [profit1, profit2]
#prices: dict {ticker: [rating, [price1, price2, ...]], ...}
#shares: dict {3: quantity, 4: quantity, 5: quantity}

threeStarVolume = 200
fourStarVolume = 400
fiveStarVolume = 800

shares = {0:1200, 3: 200, 4: 400, 5: 800}

def calculateProfits(ticker_prices, shares):
    balance  = [0] * len(next(iter(ticker_prices.values()))[1])
    
    for ticker, ops in ticker_prices.items():
        rating = ops[0]
        prices = ops[1]
        for i in range(len(balance)):
            balance[i] = balance[i] + prices[i]*shares[rating]
    profits_percentage = [0] * len(balance)
    for i in range(len(balance)):
        profits_percentage[i] = (balance[i]-balance[0])/balance[0]
    return profits_percentage

In [72]:
profits = calculateProfits(ticker_prices, shares)

profits_snp = calculateProfits(snp_prices, shares)

profits

[0.0,
 -0.0033243328564328523,
 -0.017138900964282385,
 -0.020048772365862253,
 -0.018116046571773094]

In [73]:
def getDealDays(report_date, end_date, ticker):
    panel_data = data.DataReader(ticker, 'yahoo', report_date, end_date)
    return [pd.to_datetime(d).strftime("%m/%d") for d in list(panel_data.index.values)]

In [81]:
deal_days =  getDealDays(report_date, end_date, '^GSPC')
l1, = plt.plot(deal_days,profits, label='Morningstar')
l2, = plt.plot(deal_days,profits_snp, label='SNP')
plt.legend(loc='upper right')
plt.title('Title')
plt.show()

deal_days

['08/08', '08/09', '08/10', '08/13', '08/14']

In [85]:
def getFormatedTickerPrice(ticker_prices, shares):
    s = "Ticker Quantity Open_Price Close_Price\n"
    for ticker, ops in ticker_prices.items():
        ticker = "{:<6}".format(ticker)
        rating = ops[0]
        rating = "{:<8}".format(str(shares[rating]))
        s = s + ticker + ' ' + rating
        prices = ops[1]
        open_price = "{:<10}".format( str(round(prices[0],2)) )
        close_price = "{:11}".format( str(round(prices[-1],2)) )
        s = s + ' '+  open_price +' '+ close_price
        s = s+'\n'
    return s

print (getFormatedTickerPrice(ticker_prices, shares))

Ticker Quantity Open_Price Close_Price
CCJ    800      11.11      10.74      
CMP    400      65.05      61.85      
MLM    400      205.44     202.75     
CMCSA  400      35.05      35.39      
TEF    800      8.89       8.48       
BABA   400      177.52     172.53     
EXPE   400      131.71     133.49     
GM     400      37.66      36.2       
HBI    800      18.58      18.19      
IVC    400      16.5       16.3       
MAT    400      15.52      15.56      
NCLH   400      48.93      52.02      
DIS    400      113.98     112.75     
WPP    400      79.05      77.81      
GEM    800      33.53      32.52      
GIS    800      45.22      45.8       
MDLZ   400      42.39      41.87      
PEP    400      113.49     113.1      
PG     400      82.35      81.31      
CVE    400      9.81       9.54       
ENB    800      35.77      35.51      
EPD    400      29.26      29.3       
TOT    400      64.4       60.93      
AIG    800      52.44      52.33      
BLK    400      487.05   